## 楽天トラベル地区コードAPI

In [2]:
import pandas as pd
import csv
import requests

REQUEST_URL = 'https://app.rakuten.co.jp/services/api/Travel/GetAreaClass/20131024'
APP_ID = '○○' # 自身のAPP_IDを入力

params = {
        "formatVersion":2,
        "applicationId":APP_ID
        }

res = requests.get(REQUEST_URL, params)
result = res.json()

In [3]:
result['areaClasses']['largeClasses'][0][1]['middleClasses']

[{'middleClass': [{'middleClassCode': 'hokkaido', 'middleClassName': '北海道'},
   {'smallClasses': [{'smallClass': [{'smallClassCode': 'sapporo',
        'smallClassName': '札幌'},
       {'detailClasses': [{'detailClass': {'detailClassCode': 'A',
           'detailClassName': '札幌・新札幌・琴似'}},
         {'detailClass': {'detailClassCode': 'B',
           'detailClassName': '大通公園・時計台・狸小路'}},
         {'detailClass': {'detailClassCode': 'C',
           'detailClassName': 'すすきの・中島公園'}}]}]},
     {'smallClass': [{'smallClassCode': 'jozankei', 'smallClassName': '定山渓'}]},
     {'smallClass': [{'smallClassCode': 'wakkanai',
        'smallClassName': '稚内・留萌・利尻・礼文'}]},
     {'smallClass': [{'smallClassCode': 'abashiri',
        'smallClassName': '網走・紋別・北見・知床'}]},
     {'smallClass': [{'smallClassCode': 'kushiro',
        'smallClassName': '釧路・阿寒・川湯・根室'}]},
     {'smallClass': [{'smallClassCode': 'obihiro',
        'smallClassName': '帯広・十勝'}]},
     {'smallClass': [{'smallClassCode': 'hidaka',
        

## 楽天トラベル施設検索API

In [4]:
import pandas as pd
import csv
import requests

REQUEST_URL = 'https://app.rakuten.co.jp/services/api/Travel/SimpleHotelSearch/20170426'
APP_ID = '○○' # 自身のAPP_IDを入力

def SimpleHotelSearch(path,columns_lists,middleClassCode,smallClassCode):

    df_search = pd.DataFrame()
    index = 0
    page = 1
    
    while True:
        
        params = {
            "largeClassCode":"japan",
            "middleClassCode":middleClassCode,
            "smallClassCode":smallClassCode,
            "page": page,
            "applicationId":APP_ID
        }
        
        try:
            res = requests.get(REQUEST_URL, params)
            result = res.json()
            hotels = result["hotels"]

            for hotel in hotels:
                hotel_info = hotel["hotel"][0]["hotelBasicInfo"]
                _df = pd.DataFrame(hotel_info, index=[index])
                df_search = df_search.append(_df)
                index+=1
                
        except:
            break 
        
        page+=1
            
    df_search = df_search[columns_lists]
    
    df_search.to_csv(path)
    
    return df_search

In [5]:
path = '××' # CSV保存先入力
columns_lists = ['hotelNo','hotelName','address1','address2','reviewAverage','reviewCount','reviewUrl']
middleClassCode = '××' # 楽天トラベル地区コードAPIで取得したmiddleClassCodeを入力
smallClassCode = '××' # 楽天トラベル地区コードAPIで取得したsmallClassCodeを入力

df_gobo = SimpleHotelSearch(path,columns_lists,middleClassCode,smallClassCode)

## Web スクレイピング

In [417]:
import requests
from bs4 import BeautifulSoup
import time
import random

base_url = 'https://review.travel.rakuten.co.jp/hotel/voice/{0}/?f_time=&f_keyword=&f_age=0&f_sex=0&f_mem1=0&f_mem2=0&f_mem3=0&f_mem4=0&f_mem5=0&f_teikei=&f_version=2&f_static=1&f_point=0&f_sort=0&f_next={1}'

def WebScraping(hotelNo,hotelName,last_page):
    
    df_review = pd.DataFrame()
    page = 0
    index = 0
    
    for i in range(last_page):   

        page_url = base_url.format(hotelNo, page)
        res = requests.get(page_url)
        parse_html = BeautifulSoup(res.text, 'html.parser')
        
        # 施設名取得
        hotelName = parse_html.find('a', attrs={ 'class': 'rtconds fn' }).string

        for commentBox in parse_html.find_all('div', attrs={ 'class': 'commentBox' }):

            review_info = {}
            
            # 総合点取得
            try:
                comment_Rate = commentBox.find('p', attrs={ 'class': 'commentRate' }) 
                sougou = comment_Rate.find('em').string
                rate = int(comment_Rate.find('span').string)
                review_info[sougou] = rate
            except:
                pass
            
            # 「口コミ」取得
            comment = commentBox.find('p', attrs={ 'class': 'commentSentence' }).text.strip()
            review_info['口コミ'] = comment
            
            # 「旅行の目的」「同伴者」「宿泊年月日」取得
            commentPurpose = commentBox.find('dl', attrs={ 'class': 'commentPurpose' })
            question = commentPurpose.find_all('dt')
            answer = commentPurpose.find_all('dd')
            for q,a in zip(question,answer):
                review_info[q.text] = a.text
                
            # dataframeに追加
            _df = pd.DataFrame(review_info, index=[index])
            df_review = df_review.append(_df,sort=True)
            
            # index加算
            index+=1
        
        # ページ数加算
        page += 20

        sec = random.uniform(1, 3)
        time.sleep(sec)
    
    # 施設名取得
    df_review['施設名'] = hotelName
    
    # csvファイル保存先作成
    path = '×××/{0}.csv'.format(hotelName) # csv保存先入力
    
    df_review.to_csv(path)

In [419]:
## for文回して一気に作成

hotelNo_lists = df_yura['hotelNo'].to_list()
hotelName_lists= df_yura['hotelName'].to_list()

for hotelNo, hotelName in zip(hotelNo_lists, hotelName_lists):
    
    # URL作成
    url = "https://review.travel.rakuten.co.jp/hotel/voice/{0}".format(hotelNo)
    
    # ページ取得・解析
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    
    # 最終ページ取得
    last_page = int([href for href in soup.find('ul',attrs={ 'class': 'pagingNumber'}).find('li')][-2].string)
    
    # 関数呼び出し
    WebScraping(hotelNo,hotelName,last_page)